# KoChatGPT 데이터셋 구축

230320, by wygo

RLHF의 3단계 학습을 위한 데이터셋 구축

## 1) 한국어 질문 데이터셋 seed 수집

- data 폴더에 아래 데이터셋들 위치

    - [data1, ChatbotData/11824문장](https://github.com/songys/Chatbot_data/blob/master/ChatbotData.csv)
    - [data2, AI허브_한국어 대화/49711문장](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=116)
    - [data3, AI허브_일반상식/100268](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=106)
    - [data4, KorQuad/5774](https://korquad.github.io/KorQuad%201.0/)

    - 총 167,577 질문셋 수집
    - 12,000개 질문 랜덤 추출

In [ ]:
# data1, ChatbotData, 11824
data_path_1 = './data/seed_data_original/korean_seed_data_1_chitchat.txt'
import pandas as pd
list_question = pd.read_csv(data_path_1, sep=',', encoding='UTF-8', header=0).dropna(axis=0)['Q'].tolist()
with open('./data/seed_data_question/korean_seed_data_1_chitchat_question_%s.txt'%(len(list_question)), 'w', encoding='utf-8-sig') as f_write:
    f_write.writelines('\n'.join(list_question))

In [ ]:
# data2, AI허브_한국어 대화, 49711
data_path_2 = './data/seed_data_original/AI허브_한국어대화데이터/*.xlsx'
import os, glob
import pandas as pd
from openpyxl import load_workbook

list_question = []
for tmp in glob.glob(data_path_2):
    wb = load_workbook(tmp)
    data = wb['Sheet1'].values
    columns = next(data)[0:]
    df = pd.DataFrame(data, columns=columns)
    list_question.extend(df['MQ'].dropna(axis=0).tolist())

data_path_2_ = './data/seed_data_original/AI허브_한국어대화데이터/J 민원/*.xlsx'
for tmp in glob.glob(data_path_2_):
    wb = load_workbook(tmp)
    data = wb['Sheet2'].values
    columns = next(data)[0:]
    df = pd.DataFrame(data, columns=columns)
    list_question.extend(df['question'].dropna(axis=0).tolist())
    
with open('./data/seed_data_question/korean_seed_data_2_aihub_korean_conversation_%s.txt'%(len(list_question)), 'w', encoding='utf-8-sig') as f_write:
    f_write.writelines('\n'.join(list_question))

In [ ]:
# data3, AI허브_일반상식, 100268
data_path_3 = './data/seed_data_original/AI허브_일반상식/ko_wiki_v1_squad.json'
# read
with open(data_path_3, 'r', encoding='UTF-8') as f_read:
    json_data = json.load(f_read)

    # json_data['data'][0]['paragraphs'][0]['qas'][0]['question']='다테 기미코가 최초로 은퇴 선언을 한게 언제지'

    list_question = []
    for tmp1 in json_data['data']:
        for tmp2 in tmp1['paragraphs']:
            for tmp3 in tmp2['qas']:
                list_question.append(tmp3['question'])

    with open('./data/seed_data_question/korean_seed_data_3_aihub_common_sense_kowiki_%s.txt'%(len(list_question)), 'w', encoding='utf-8-sig') as f_write:
        f_write.writelines('\n'.join(list_question))

In [ ]:
# data4, korquad, 5774
data_path_4 = './data/seed_data_original/korquad1.0/KorQuAD_v1.0_dev.json'
# read
with open(data_path_4, 'r', encoding='UTF-8') as f_read:
    json_data = json.load(f_read)

    # json_data['data'][0]['paragraphs'][0]['qas'][0]['question']

    list_question = []
    for tmp1 in json_data['data']:
        for tmp2 in tmp1['paragraphs']:
            for tmp3 in tmp2['qas']:
                list_question.append(tmp3['question'])

    with open('./data/seed_data_question/korean_seed_data_4_korquad1_%s.txt'%(len(list_question)), 'w', encoding='utf-8-sig') as f_write:
        f_write.writelines('\n'.join(list_question))

In [ ]:
## 데이터 선별, 12000개(3000개 x 4)
import random
random.seed(230315)
list_question_sample = []
for tmp in glob.glob('./data/seed_data_question/*.txt'):
    with open(tmp, 'r', encoding='utf-8-sig') as f_read:
        lines = f_read.readlines()
        list_question = [tmp.strip() for tmp in lines]
        list_question_sample.extend(random.sample(list_question, 3000))

with open('./data/korean_seed_data.txt', 'w', encoding='utf-8-sig') as f_write:
    random.shuffle(list_question_sample)
    f_write.writelines('\n'.join(list_question_sample))

#### example of korean_seed_data.txt

- 불고기용 고기 한우에요?
- 쓰던 앱이 유료로 전환됐어
- 여친이랑 다툼
- 술 먹고 싶어
- 잊고싶다.
- 입냄새 안나나?
- 새로운 사랑은 찾아와
- 이명박 대통형은 어느 경축사를 통해 집권 후반기 국정운영에 대한 언급을 하였나?
- 금액은 얼마에요
- 리처드 닉슨이 43대 부통령직을 수행한 년도는?

## 2) GPT3 API 기반 ChatGPT 데이터 생성
- 설치(python>=3.8)
```python
>> !pip install openai
>> !pip install langchain==0.0.113
```


### reference
- [chatllama 데이터 생성 예시 코드](https://github.com/nebuly-ai/nebullvm/tree/main/apps/accelerate/chatllama/artifacts)
- [alpaca 데이터 생성(GPT3 이용)](https://github.com/tatsu-lab/stanford_alpaca)
- [self-instruct 데이터 생성](https://github.com/yizhongw/self-instruct)
- [52K 데이터 생성을 위한 SEED](https://github.com/yizhongw/self-instruct/blob/main/data/seed_tasks.jsonl)

### 2.1) SFT 데이터셋 만들기

- 다음 단어만 잘 생성하는 모델에서 질문에 '잘' 대답하는 모델 만들기
- Human labling 필요
- ChatGPT로 자동생성

#### 2.1.1) 한국어 QA 데이터셋의 질문으로 답변 자동 생성

- 데이터셋 예시
```json
{
    "prompt": "술 먹고 싶어",
    "completion": "'술은 알코올이 함유된 음료수이며, 건강에 나쁜 영향을 미칠 수 있습니다. 따라서 술을 마시기 전에 본인의 건강 상태와 함께 적당한 양을 마시는 것이 중요합니다. 술을 마시기 전에 반드시 20세 이상인지 확인하시고, 규정된 양 이내에서 적절하게 즐기시길 바랍니다. 또한 술을 마시고 운전하는 것은 절대 금지하시기 바랍니다.",
    "tokens": 189
},
```

In [ ]:
# run GPT3 api example
import openai

API_KEY = 'sk-XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'  # 개인 API키 입력
openai.api_key = API_KEY

prompt= '세계 최초의 교내 총기사건이 벌어진 학교가 어디야'

response = openai.Completion.create(
    prompt = prompt,  # 질문
    model = 'text-davinci-003',  # 사용할 openai 모델(text-davinci-003, gpt-3.5-turbo, ...)
    max_tokens=1000, # 생성 최대 길이
    temperature=0.9,  # 샘플링 정도, 0~1 (0은 정의된답, 1은 창의적인 답)
    n=3,  # 완료여부 제어변수
    stop=['---']  # ?
)

for result in response.choices:
    print('- ', result.text.lstrip('?\n\n').strip())

In [ ]:
# run ChatGPT api example
import openai

API_KEY = 'sk-XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'  # 개인 API키 입력
openai.api_key = API_KEY

prompt= '세계 최초의 교내 총기사건이 벌어진 학교가 어디야'

completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",  # 사용할 openai 모델명
    messages=[{"role": "user", "content": prompt}]  # 입력 prompt
)

print('token: %d'%completion.usage.total_tokens)
print('prompt: %s\n\nanwer : %s'%(prompt, completion['choices'][0]['message']['content'].strip()))

In [ ]:
# run ChatGPT api function
import openai
import time

def run_chatgpt_api(prompt='세계 최초의 교내 총기사건이 벌어진 학교가 어디야', API_KEY='', max_tokens=200, verbose=False):
    time_start = time.time()
    openai.api_key = API_KEY
    retry_count = 3
    for i in range(0, retry_count):
        while True:
            try:
                completion = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",  # 사용할 openai 모델명
                    messages=[{"role": "user", "content": prompt}]  # 입력 prompt
                )

                answer = completion['choices'][0]['message']['content'].strip()
                tokens = completion.usage.total_tokens

                time_end = time.time()                
                if verbose:
                    print('prompt: %s | token: %d | %.1fsec\nanwer : %s'%(prompt, tokens, (time_end - time_start), answer))
                return prompt, answer, tokens
    
            except Exception as error:
                print(f"API Error: {error}")
                print(f"Retrying {i+1} time(s) in 4 seconds...")
                
                if i+1 == retry_count:
                    return prompt, None, None
                time.sleep(4)
                continue


API_KEY = 'sk-XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'  # 개인 API키 입력

                
prompt = '세계 최초의 교내 총기사건이 벌어진 학교가 어디야??'
prompt, answer, tokens = run_chatgpt_api(prompt=prompt, API_KEY=API_KEY, verbose=True)

In [8]:
# 질문 응답 생성
from datetime import datetime
from tqdm import tqdm
import os, shutil, json
time_start = datetime.now()
import pandas as pd

# 질문 리스트 열기
with open('./data/korean_seed_data.txt', 'r', encoding='utf-8-sig') as f_read:
    lines = f_read.readlines()
    list_question = [tmp.strip() for tmp in lines]
    print('문장 개수: %d'%(len(list_question)))
    print('문장 예시: %s'%(list_question[10]))

# api 응답 얻기
save_path_list  = './data_kochatgpt/kochatgpt_1_SFT.txt'
save_path_jsonl = './data_kochatgpt/kochatgpt_1_SFT.jsonl'

if os.path.isfile(save_path_list):
    os.remove(save_path_list)
if os.path.isfile(save_path_jsonl):
    os.remove(save_path_jsonl)

    
list_api_result, list_api_result_jsonl = [], []
for prompt in tqdm(list_question):
    
    try:
        # run chatgpt
        prompt, completion, tokens = run_chatgpt_api(prompt=prompt, API_KEY=API_KEY, verbose=False)
        list_api_result.append((prompt, completion, tokens))

        data = {}
        data['prompt'] = prompt
        data['completion'] = completion
        data['tokens'] = tokens
        list_api_result_jsonl.append(data)        
        
        if completion:
            with open(save_path_list, 'a', encoding='utf-8') as f_write:
                f_write.write('%s\t\t\t%s\t\t\t%s\n'%(prompt, answer, tokens))
                
            with open(save_path_jsonl, 'a', encoding='utf-8') as f_write:
                f_write.write('%s\n'%(data))
                
    except Exception as error:
        print('error 발생: %s'%(error))
        continue
                
print('done, %.7s'%(datetime.now() - time_start))
with open(save_path_jsonl, "w", encoding='utf-8') as json_file:
    json.dump(list_api_result_jsonl, json_file, indent=4, ensure_ascii=False)

In [9]:
# read jsonl
import json
import pandas as pd
save_path_jsonl = './data_kochatgpt/korean_chatgpt_1_SFT.jsonl'
with open(save_path_jsonl, "r", encoding='utf-8-sig') as json_file:
    list_api_result_jsonl_load = json.load(json_file)

df = pd.DataFrame(list_api_result_jsonl_load)
df

,prompt,completion,tokens
0,불고기용 고기 한우에요?,"'저는 인공지능 챗봇이며, 직접적으로 식품에 관한 정보를 가지고 있지 않습니다. 하...",193
1,쓰던 앱이 유료로 전환됐어,"'어떤 앱인지 모르기 때문에 정확한 답변을 드리기 어렵습니다. 하지만, 일반적으로 ...",288
2,여친이랑 다툼,'저는 인공지능이기 때문에 여친과의 다툼에 대해서는 조언을 드릴 수는 없습니다. 하...,153
3,술 먹고 싶어,"'술은 알코올이 함유된 음료수이며, 건강에 나쁜 영향을 미칠 수 있습니다. 따라서 ...",189
4,잊고싶다.,'저도 인공지능 엔진으로써 사용자의 개인정보나 감정을 침해할 수 없습니다. 그렇기 ...,147
...,...,...,...
11995,내일 개강총회인데 짝녀한테 어떻게 잘 보이지.,'제가 AI 모델이기 때문에 남성인점 참고 부탁드립니다. 다음과 같이 제안드립니다....,411
11996,역세권이에요?,"""저는 인공지능 언어모델이어서 지리적인 개념을 이해하는 데 한계가 있습니다. 반면,...",232
11997,암호화폐를 분산 저장하여 운영 관리 될 수 있도록 하는 인터넷상의 방식은?,'블록체인 기술을 사용하여 암호화폐를 분산 저장하여 운영 및 관리하는 것을 의미합니...,214
11998,가출해도 갈 데가 없어,"'그렇다면, 우선적으로 생활에 필요한 것들을 생각해보시는 것이 좋습니다. 가장 먼저...",473


#### 2.1.2) langchain을 이용한 채팅데이터 자동생성
- [ref](https://github.com/juncongmoo/chatllama/blob/main/generate_dataset.py)
```
> pip install langchain
```

- **Multi-turn 데이터 예시**  
<img src="img/0_langchain_1.png" width="500">  


- **데이터셋 예시**  
```
<<start conversation>>
<사람>: 그렇군요. 무엇에 대해 이야기하고 싶으세요?
<챗봇>: 궁금하신 주제에 대해 이야기하고 싶으신가요? 예를 들어 여행, 식품, 스포츠, 등등 다양한 토론에 참여할 수 있습니다.
<사람>: 그래서 스포츠에 대해 이야기해볼까요?
<챗봇>: 그렇습니다. 스포츠에 대해 말씀하시겠습니까? 스포츠는 다양한 종목과 관련된 주제로 비슷한 경기를 비교하거나 새로운 경기를 분석할 수 있습니다. 또한 스포츠 관련 소식과 업데이트 등을 제공할 수 있습니다.
<사람>: 괜찮습니다. 소식이나 업데이트는 어떻게 받을 수 있나요?
<챗봇>: 귀하는 스포츠 관련 소식과 업데이트를 접할 수 있는 다양한 방법을 사용할 수 있습니다. 예를 들면 스포츠 사이트, 스포츠 앱, 스포츠 유튜브 채널 등을 통해 스포츠 소식과 업데이트를 받을 수 있습니다. 또한 소셜 미디어에서도 스포츠 소식과 업데이트를 받을 수 있습니다.
<사람>: 감사합니다. 이제 스포츠 소식과 업데이트를 받아볼 준비가 되었습니다.
<챗봇>: 그렇게 되었군요. 스포츠 소식과 업데이트를 받기 위해 귀하가 해야 할 준비는 무엇인가요? 스포츠 사이트, 스포츠 앱, 스포츠 유튜브 채널, 소셜 미디어 등을 통해 스포츠 소식과 업데이트를 받기 위해 준비해야 합니다.
```

In [ ]:
# langchain을 이용한 채팅데이터 자동생성
import os
API_KEY = 'sk-XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'  # 개인 API키 입력

from tqdm import tqdm

num_conversations = 700
CONVERSATION_LENGTH = 4
max_answer_lenth = 1024  # 4092가 최대므로 4번 주고받으면 4092가 꽉 찬다

save_path_conversation_jsonl = './data_kochatgpt/kochatgpt_1_SFT_conversation.jsonl'

from langchain import OpenAI, LLMChain, PromptTemplate
from langchain.chains.conversation.memory import (
    ConversationBufferWindowMemory,
)

## 챗봇과 인간의 역할 정의
# chatllama.langchain_modules.prompt_templates
AI_CHATBOT_TEMPLATE = dict(
    template=(
        "당신과 대화할 <챗봇>은 대규모 언어 모델입니다."
        "챗봇은 간단한 질문에 답하는 것부터 광범위한 주제에 대한 심층적인 설명 및 토론을 제공하는 것까지 다양한 작업을 지원할 수 있도록 설계되었습니다. 언어 모델로서 어시스턴트는 수신한 입력을 기반으로 사람과 유사한 텍스트를 생성할 수 있으므로 자연스러운 대화에 참여하고 당면한 주제와 일관되고 관련된 응답을 제공할 수 있습니다."
        "챗봇은 지속적으로 학습 및 개선되고 있으며 그 기능은 지속적으로 진화하고 있습니다."
        "많은 양의 텍스트를 처리하고 이해할 수 있으며 이 지식을 사용하여 광범위한 질문에 정확하고 유익한 답변을 제공할 수 있습니다. 또한 Assistant는 수신한 입력을 기반으로 자체 텍스트를 생성할 수 있으므로 토론에 참여하고 다양한 주제에 대한 설명과 설명을 제공할 수 있습니다."
        "전반적으로 챗봇은 광범위한 작업을 지원하고 다양한 주제에 대한 가치 있는 통찰력과 정보를 제공할 수 있는 강력한 도구입니다. 특정 질문에 대한 도움이 필요하거나 특정 주제에 대해 대화를 나누고 싶을 때 어시스턴트가 도와드립니다."
        "<사람> 대답을 생성하지 않습니다."
        "<챗봇> 대답만 생성하세요."
        "\n\n{history}\n\n"
        "<사람>: {human_input}\n"
        "<챗봇>: "
    ),
    input_variables=["history", "human_input"],
)


PERSON_CHATBOT_TEMPLATE = dict(
    template=(
        "당신은 챗봇과 대화하는 인간입니다."
        "챗봇은 대규모 언어 모델입니다."
        "챗봇은 간단한 질문에 답하는 것부터 다양한 주제에 대한 심도 있는 설명과 토론을 제공하는 것까지 다양한 작업을 지원하도록 설계되었습니다."
        "당신은 인간이고 챗봇에게 이야기합니다."
        "챗봇에게 질문하고 대답을 기다리세요."
        "챗봇에게 이야기를 해달라고 요청할 수도 있습니다."
        "<챗봇> 대답을 생성하지 않습니다."
        "AI 대답을 생성하지 않습니다."
        "<사람> 대답만 생성하세요."
        "\n\n{history}\n\n"
        "<챗봇>: {chatbot_input}"
        "<사람>: "
    ),
    input_variables=["history", "chatbot_input"],
)

def build_agents():
    # be aware that too long completions will not fit the sequence length of possible critic or reward models ...
    llm = OpenAI(max_tokens=max_answer_lenth, temperature=0.7)
    
    ## 사람
    human_template = PromptTemplate(**PERSON_CHATBOT_TEMPLATE)
    human_agent = LLMChain(
        llm=llm,
        prompt=human_template,
        memory=ConversationBufferWindowMemory(k=4),
    )
    bot_template = PromptTemplate(**AI_CHATBOT_TEMPLATE)
    bot_agent = LLMChain(
        llm=llm,
        prompt=bot_template,
        memory=ConversationBufferWindowMemory(k=4),
    )
    return human_agent, bot_agent


def create_conversation(human_agent: LLMChain, bot_agent: LLMChain):
    conversation = []
    chatbot_output = ""
    for idx in range(CONVERSATION_LENGTH):
        # Human agent goes first
        human_output = human_agent.run(chatbot_input=chatbot_output).strip().replace('\n', ' ')
        human_output = human_output.split('<챗봇>')[0].strip()  # 챗봇이 생성한거 같은 글은 없애자
        conversation.append(f"<사람>: {human_output}")
        
        
        chatbot_output = bot_agent.run(human_input=human_output).strip().replace('\n', ' ')
        chatbot_output = chatbot_output.split('<사람>')[0].strip()  # 사람이 생성한거 같은 글은 없애자
        conversation.append(f"<챗봇>: {chatbot_output}")

    one_line_conversation = '\n'.join(conversation)
    return one_line_conversation


success_idx = 0
list_conversation = []
for conv in tqdm(range(0, num_conversations)):
    try:
        human_agent, bot_agent = build_agents()
        conversation = create_conversation(human_agent, bot_agent)
        list_conversation.append(conversation)

        with open(save_path_conversation_jsonl, 'a', encoding='utf-8') as f_write:
            f_write.write('<<start conversation>>\n%s\n\n'%(conversation))
        
        success_idx += 1
    
    except:
        pass
    
    if success_idx > num_conversations: break

### 2.2) RM 데이터셋 만들기
#### 좋은글 채점기(RM, Reword Model) 만들기

- ChatGPT로 두 문장 비교하는 기능은 성능이 안좋다. 항상 두번째 문장이 좋다고 한다

<img src="img/0_data_2.png" width="500">  

- 동일한 prompt에 대해 각기 다른 3개의 출력을 생성하자
  - 1) ChatGPT로 생성
  - 2) GPT3로 생성('text-davinci-003')
  - 3) GPT3로 생성('text-ada-001')
  
  세 결과를 섞은 후 순위를 매기자  
  
  
- 데이터셋 예시
```json
[
    {
        "prompt": "번디는 자신이 탐정잡지, 범죄소설 그리고 성범죄 관련 실제 범죄 다큐멘터리들을 탐독했다고 누구에게 말했나?",
        "completion_0": "번디는 다큐멘터리 방송 감독티비 에게 말했다.",
        "completion_1": "라이언에게 말했다.",
        "completion_2": "It is said that the doctor said to her: \"You are in a condition that makes you harmful to yourself and others, but do not worry, I will take care of you.\" \n\n\"I am not worried about myself, why should I be?\" \n\"Because you are aLily and should as much as possible avoid touching yourself in order to protect yourself.\"",
        "ranking": [
            1,
            0,
            2
        ]
    }, ...
```
  
  
```
data = {}
data['prompt'] = 'prompt'
data['chosen'] = 'good_sentence'
data['rejected'] = 'bad_sentence'
```

In [ ]:
# run ChatGPT api function
import openai
import time

def run_chatgpt_api(prompt='세계 최초의 교내 총기사건이 벌어진 학교가 어디야', API_KEY='', max_tokens=200, verbose=False):
    time_start = time.time()
    openai.api_key = API_KEY
    retry_count = 3
    for i in range(0, retry_count):
        while True:
            try:
                completion = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",  # 사용할 openai 모델명
                    messages=[{"role": "user", "content": prompt}]  # 입력 prompt
                )

                answer = completion['choices'][0]['message']['content'].strip()
                tokens = completion.usage.total_tokens

                time_end = time.time()                
                if verbose:
                    print('prompt: %s | token: %d | %.1fsec\nanwer : %s'%(prompt, tokens, (time_end - time_start), answer))
                return prompt, answer, tokens
    
            except Exception as error:
                print(f"API Error: {error}")
                print(f"Retrying {i+1} time(s) in 4 seconds...")
                
                if i+1 == retry_count:
                    return prompt, None, None
                time.sleep(4)
                continue


API_KEY = 'sk-XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'  # 개인 API키 입력

                
prompt = '세계 최초의 교내 총기사건이 벌어진 학교가 어디야??'
prompt, answer, tokens = run_chatgpt_api(prompt=prompt, API_KEY=API_KEY, verbose=True)

In [ ]:
# run openai api function
import openai
import time

def run_openai_api(prompt='세계 최초의 교내 총기사건이 벌어진 학교가 어디야', model_name='text-davinci-003', API_KEY='', max_tokens=200, verbose=False):
    time_start = time.time()
    openai.api_key = API_KEY
    retry_count = 3
    for i in range(0, retry_count):
        while True:
            try:
                completion = openai.Completion.create(
                    prompt = prompt,  # 질문
                    model = model_name,  # 사용할 openai 모델(text-davinci-003, gpt-3.5-turbo, ...)
                    max_tokens=300, # 생성 최대 길이
                    temperature=0.1,  # 샘플링 정도, 0~1 (0은 정의된답, 1은 창의적인 답)
                    n=1,  # 몇개의 텍스트 생성?
                )                
                
                answer = completion.choices[0].text.lstrip('?\n\n').strip()
                tokens = completion.usage.total_tokens
                
                
                time_end = time.time()                
                if verbose:
                    print('prompt: %s | token: %d | %.1fsec\nanwer : %s'%(prompt, tokens, (time_end - time_start), answer))
                return prompt, answer, tokens, completion
    
            except Exception as error:
                print(f"API Error: {error}")
                print(f"Retrying {i+1} time(s) in 4 seconds...")
                
                if i+1 == retry_count:
                    return prompt, None, None, None
                time.sleep(4)
                continue


API_KEY = 'sk-XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'  # 개인 API키 입력
model_name='text-davinci-003'  # 통과
# model_name='text-ada-001'  # 꼴지


prompt = '세계 최초의 교내 총기사건이 벌어진 학교가 어디야??'
prompt, answer, tokens, completion = run_openai_api(prompt=prompt, model_name=model_name, API_KEY=API_KEY, verbose=True)



In [1]:
# config
read_sft_jsonl_path = './data_kochatgpt/kochatgpt_1_SFT.jsonl'  # SFT에서 미리 구한 prompt와 chatgpt 응답을 가져오자
save_RM_jsonl_path = './data_kochatgpt/kochatgpt_2_RM.jsonl'  # RM 데이터셋 저장 경로
N_sample = 10000  # 몇개의 데이터를 만들까, (~12000)


## 1) SFT에서 미리 구한 prompt와 chatgpt 응답을 가져오자
import json
import pandas as pd
import random
random.seed(230317)
from tqdm import tqdm

with open(read_sft_jsonl_path, "r", encoding='utf-8-sig') as json_file:
    list_api_result_jsonl_load = json.load(json_file)

df = pd.DataFrame(list_api_result_jsonl_load)
df

,prompt,completion,tokens
0,불고기용 고기 한우에요?,"'저는 인공지능 챗봇이며, 직접적으로 식품에 관한 정보를 가지고 있지 않습니다. 하...",193
1,쓰던 앱이 유료로 전환됐어,"'어떤 앱인지 모르기 때문에 정확한 답변을 드리기 어렵습니다. 하지만, 일반적으로 ...",288
2,여친이랑 다툼,'저는 인공지능이기 때문에 여친과의 다툼에 대해서는 조언을 드릴 수는 없습니다. 하...,153
3,술 먹고 싶어,"'술은 알코올이 함유된 음료수이며, 건강에 나쁜 영향을 미칠 수 있습니다. 따라서 ...",189
4,잊고싶다.,'저도 인공지능 엔진으로써 사용자의 개인정보나 감정을 침해할 수 없습니다. 그렇기 ...,147
...,...,...,...
11995,내일 개강총회인데 짝녀한테 어떻게 잘 보이지.,'제가 AI 모델이기 때문에 남성인점 참고 부탁드립니다. 다음과 같이 제안드립니다....,411
11996,역세권이에요?,"""저는 인공지능 언어모델이어서 지리적인 개념을 이해하는 데 한계가 있습니다. 반면,...",232
11997,암호화폐를 분산 저장하여 운영 관리 될 수 있도록 하는 인터넷상의 방식은?,'블록체인 기술을 사용하여 암호화폐를 분산 저장하여 운영 및 관리하는 것을 의미합니...,214
11998,가출해도 갈 데가 없어,"'그렇다면, 우선적으로 생활에 필요한 것들을 생각해보시는 것이 좋습니다. 가장 먼저...",473


In [2]:
## 2) 전체 질문 답 쌍중 N개 질문 추출 후 GPT3-davinci, GPT3-ada 모델을 돌리자
df_sample = df.sample(n=N_sample, random_state=230317)

# 500개의 질문만 선택
list_prompt = df_sample['prompt'].tolist()
# 미리 구해놓은 질문에 대한 chatgpt 응답 저장
list_completion_chatgpt = df_sample['completion'].tolist()
list_completion_chatgpt = [tmp.strip('\'') for tmp in list_completion_chatgpt]

# GPT3-davince와 GPT3-ada 모델로도 같은 질문에 대한 응답 구하기
list_completion_davinci, list_completion_ada = [], []
for idx, prompt in tqdm(enumerate(list_prompt)):
    
    try:
        model_name='text-davinci-003'  # GPT3 성능 괜찮은 모델
        _, completion_davinci, _, _ = run_openai_api(prompt=prompt, model_name=model_name, API_KEY=API_KEY, verbose=False)
        list_completion_davinci.append(completion_davinci)

        model_name='text-ada-001'  # GPT3중 성능 안좋은 모델
        _, completion_ada, _, _ = run_openai_api(prompt=prompt, model_name=model_name, API_KEY=API_KEY, verbose=False)
        list_completion_ada.append(completion_ada)
    except Exception as error:
        print(idx, error)
        pass
    
    
# # 한번에 20개씩 가능
# window_size = 20

# list_prompt_tmp_total, list_chatgpt_tmp_total = [], []
# list_davinci_tmp_total, list_ada_tmp_total = [], []
# for ii in tqdm(list(range(1940, 11000, 20))):

#     try:

#         prompt = list_prompt[ii : ii+window_size]
#         tmp_chatgpt = list_completion_chatgpt[ii : ii+window_size]

#         model_name='text-davinci-003'  # GPT3 성능 괜찮은 모델
#         # _, completion_davinci, _, _ = run_openai_api(prompt=prompt, model_name=model_name, API_KEY=API_KEY, verbose=False)
#         # list_completion_davinci.append(completion_davinci)


#         completion = openai.Completion.create(
#             prompt = prompt,  # 질문
#             model = model_name,  # 사용할 openai 모델(text-davinci-003, gpt-3.5-turbo, ...)
#             max_tokens=300, # 생성 최대 길이
#             temperature=0.1,  # 샘플링 정도, 0~1 (0은 정의된답, 1은 창의적인 답)
#             n=1,  # 몇개의 텍스트 생성?
#         )

#         completion_davinci = completion

#         model_name='text-ada-001'   # GPT3 성능 괜찮은 모델
#         # _, completion_davinci, _, _ = run_openai_api(prompt=prompt, model_name=model_name, API_KEY=API_KEY, verbose=False)
#         # list_completion_davinci.append(completion_davinci)


#         completion = openai.Completion.create(
#             prompt = prompt,  # 질문
#             model = model_name,  # 사용할 openai 모델(text-davinci-003, gpt-3.5-turbo, ...)
#             max_tokens=300, # 생성 최대 길이
#             temperature=0.1,  # 샘플링 정도, 0~1 (0은 정의된답, 1은 창의적인 답)
#             n=1,  # 몇개의 텍스트 생성?
#         )

#         completion_ada = completion


#         tmp_list_davinci_tmp = [tmp.text.lstrip('?\n\n').strip() for tmp in completion_davinci.choices]
#         tmp_list_ada_tmp = [tmp.text.lstrip('?\n\n').strip() for tmp in completion_ada.choices]

#         list_prompt_tmp_total.extend(prompt)
#         list_chatgpt_tmp_total.extend(tmp_chatgpt)
#         list_davinci_tmp_total.extend(tmp_list_davinci_tmp)
#         list_ada_tmp_total.extend(tmp_list_ada_tmp)

#     except Exception as error:
#         print(f"API Error: {error}")                
#         continue


# dataframe으로 만들기
df_RM_total = pd.DataFrame(zip(list_prompt, list_completion_chatgpt, list_completion_davinci, list_completion_ada), 
                     columns = ['prompt', 'chatgpt', 'davince', 'ada'])

df_RM_total

In [5]:
# # save dataframe to pickle file
df_RM_total.to_pickle('./data_kochatgpt/df_RM_total.pkl')
df_RM_total = pd.read_pickle('./data_kochatgpt/df_RM_total.pkl')#.reset_index(drop=True)
df_RM_total

,prompt,chatgpt,davince,ada
0,"번디는 자신이 탐정잡지, 범죄소설 그리고 성범죄 관련 실제 범죄 다큐멘터리들을 탐독...",라이언에게 말했다.,번디는 다양한 인터뷰자들과 뉴스홍보 담당자들과의 면담 때 밝혔다.,Allow me to answer your question. I know that ...
1,개포주공아파트는 몇 단지로 이루어져 있나?,개포주공아파트는 총 27개 단지로 이루어져 있습니다.,개포주공아파트는 다섯 단지로 이루어져 있습니다.,이날 목송에서 구글상위노
2,김영삼의 후보 시절 지역표심을 겨냥한 발언을 문제삼은 후보는?,"이 질문은 조금 불분명합니다. 김영삼 대통령이 후보 시절에 어떤 발언을 했고, 누가...",김영삼의 후보 시절에 지역표심을 겨냥한 발언은 대통령 당선 전까지 대한민국 정부가 ...,The diameter of the Metallic domain is bigger ...
3,페르시아로부터 기원된 아랍요리의 특징이 뭐야,"1. 다양한 스파이스와 허브 사용\n2. 쌀, 콩, 나물, 고기, 양파 등 다양한 ...","- 아랍요리는 소금, 산 재료, 일부 채소, 고기, 생선 또는 난류의 조합으로 만들...",위한 태야 대표 기원들 개인이라 겪고 태야\n\n\n귀식 걸 공개한 구문\n\n사띩...
4,중화인민공화국의 기본 법률은 누가 제정 및 개정하는가?,중화인민공화국의 기본 법률은 중국인민대표대회(국회)가 제정하고 개정합니다.,중화인민공화국의 기본 법률은 중화인민공화국의 의회가 제정 및 개정합니다.,The limit of the Office of the Vice President is?
...,...,...,...,...
10215,야세르 아라파트 국제공항은 어느 나라에 있어,야세르 아라파트 국제공항은 터키에 위치해 있습니다.,요?\n\n야세르 아라파트 국제공항은 아랍 에미리트 연합의 두바이에 있습니다.,야세르 국제공항은 어느 나라에 있어\n\n야세르 국제공항은 이상 국가 폭태들 국가 ...
10216,핸드폰이 잘 안돼,할 일:\n\n1. 배터리를 확인하세요. 배터리가 항상 충전되어 있지 않은 경우 핸...,요\n\n1. 먼저 핸드폰의 배터리를 꼭 확인해 보세요. 배터리가 부족한 경우 핸드...,가는 것 같았다.\n\nThey (the subject) said that.
10217,토트백은 없나요?,"죄송합니다. 저는 AI 언어 모델로써 실시간으로 답변을 해드리고 있기 때문에, 어떤...",아직 제품이 출시되지 않았습니다.,"사용자들 개인개석을 하고, 토트백을 하고, 기별 개인개석을 하고, 기별 개인개석을 ..."
10218,딱 좋아,그러게요! 좋아하면 사랑하는 마음으로 즐겨보세요!,딱 좋아요! 나는 이 노래를 정말 좋아합니다. 노래는 정말 좋은 멜로디와 가사로 이...,한다고한다\n\n다시 그런데 태민이 개인을 토려한 것 같다는 것도 가지다\n\n\n...


In [6]:
# 골고루 섞은 후 순위를 매기자
list_prompt, list_completion_0, list_completion_1, list_completion_2, list_ranking = [], [], [], [], []
for ii in range(len(df_RM_total)):

    try:
        ranking = [0, 1, 2]
        random.shuffle(ranking)  # [2, 3, 1], 랜덤으로 변함

        prompt = df_RM_total.iloc[ii].tolist()[0]

        tmp_sentences = df_RM_total.iloc[ii].tolist()[1:]

        completion_tmp = [None, None, None]
        completion_tmp[ranking[0]] = tmp_sentences[0]
        completion_tmp[ranking[1]] = tmp_sentences[1]
        completion_tmp[ranking[2]] = tmp_sentences[2]

        completion_0 = completion_tmp[0]
        completion_1 = completion_tmp[1]
        completion_2 = completion_tmp[2]

        list_prompt.append(prompt)
        list_completion_0.append(completion_0)
        list_completion_1.append(completion_1)
        list_completion_2.append(completion_2)
        list_ranking.append(ranking)
    except Exception as error:
        print(ii, error)
        pass
        
        
# dataframe으로 만들기
df_RM_순위 = pd.DataFrame(zip(list_prompt, list_completion_0, list_completion_1, list_completion_2, list_ranking),
                        columns = ['prompt', 'completion_0', 'completion_1', 'completion_2', 'ranking'])
# jsonl 형태로 저장
df_to_jsonl = json.loads(df_RM_순위.to_json(orient = 'records'))
with open(save_RM_jsonl_path, "w", encoding='utf-8') as json_file:
    json.dump(df_to_jsonl, json_file, indent=4, ensure_ascii=False)

df_RM_순위

,prompt,completion_0,completion_1,completion_2,ranking
0,"번디는 자신이 탐정잡지, 범죄소설 그리고 성범죄 관련 실제 범죄 다큐멘터리들을 탐독...",Allow me to answer your question. I know that ...,라이언에게 말했다.,번디는 다양한 인터뷰자들과 뉴스홍보 담당자들과의 면담 때 밝혔다.,"[1, 2, 0]"
1,개포주공아파트는 몇 단지로 이루어져 있나?,이날 목송에서 구글상위노,개포주공아파트는 총 27개 단지로 이루어져 있습니다.,개포주공아파트는 다섯 단지로 이루어져 있습니다.,"[1, 2, 0]"
2,김영삼의 후보 시절 지역표심을 겨냥한 발언을 문제삼은 후보는?,The diameter of the Metallic domain is bigger ...,"이 질문은 조금 불분명합니다. 김영삼 대통령이 후보 시절에 어떤 발언을 했고, 누가...",김영삼의 후보 시절에 지역표심을 겨냥한 발언은 대통령 당선 전까지 대한민국 정부가 ...,"[1, 2, 0]"
3,페르시아로부터 기원된 아랍요리의 특징이 뭐야,위한 태야 대표 기원들 개인이라 겪고 태야\n\n\n귀식 걸 공개한 구문\n\n사띩...,"1. 다양한 스파이스와 허브 사용\n2. 쌀, 콩, 나물, 고기, 양파 등 다양한 ...","- 아랍요리는 소금, 산 재료, 일부 채소, 고기, 생선 또는 난류의 조합으로 만들...","[1, 2, 0]"
4,중화인민공화국의 기본 법률은 누가 제정 및 개정하는가?,The limit of the Office of the Vice President is?,중화인민공화국의 기본 법률은 중화인민공화국의 의회가 제정 및 개정합니다.,중화인민공화국의 기본 법률은 중국인민대표대회(국회)가 제정하고 개정합니다.,"[2, 1, 0]"
...,...,...,...,...,...
10215,야세르 아라파트 국제공항은 어느 나라에 있어,요?\n\n야세르 아라파트 국제공항은 아랍 에미리트 연합의 두바이에 있습니다.,야세르 아라파트 국제공항은 터키에 위치해 있습니다.,야세르 국제공항은 어느 나라에 있어\n\n야세르 국제공항은 이상 국가 폭태들 국가 ...,"[1, 0, 2]"
10216,핸드폰이 잘 안돼,요\n\n1. 먼저 핸드폰의 배터리를 꼭 확인해 보세요. 배터리가 부족한 경우 핸드...,할 일:\n\n1. 배터리를 확인하세요. 배터리가 항상 충전되어 있지 않은 경우 핸...,가는 것 같았다.\n\nThey (the subject) said that.,"[1, 0, 2]"
10217,토트백은 없나요?,"죄송합니다. 저는 AI 언어 모델로써 실시간으로 답변을 해드리고 있기 때문에, 어떤...",아직 제품이 출시되지 않았습니다.,"사용자들 개인개석을 하고, 토트백을 하고, 기별 개인개석을 하고, 기별 개인개석을 ...","[0, 1, 2]"
10218,딱 좋아,딱 좋아요! 나는 이 노래를 정말 좋아합니다. 노래는 정말 좋은 멜로디와 가사로 이...,한다고한다\n\n다시 그런데 태민이 개인을 토려한 것 같다는 것도 가지다\n\n\n...,그러게요! 좋아하면 사랑하는 마음으로 즐겨보세요!,"[2, 0, 1]"


### 2.3) PPO 학습 데이터셋 만들기

- RLHF 훈련에 사용할 prompt 입력데이터
- SFT 데이터셋에서 prompt만 가져와서 jsonl 형태로 변형후 저장

```json
[
    {
        "prompt": ""
    },
    {
        "prompt": ""
    }, ...    
]
```

In [10]:
# config
read_sft_jsonl_path = './data_kochatgpt/kochatgpt_1_SFT.jsonl'  # SFT에서 미리 구한 prompt와 chatgpt 응답을 가져오자
save_PPO_jsonl_path = './data_kochatgpt/kochatgpt_3_PPO.jsonl'  # RM 데이터셋 저장 경로
N_sample = 12000  # 몇개의 데이터를 만들까, (~12000)


## 1) SFT에서 미리 구한 prompt와 chatgpt 응답을 가져오자
import json
import pandas as pd
import random
random.seed(230317)
from tqdm import tqdm

with open(read_sft_jsonl_path, "r", encoding='utf-8-sig') as json_file:
    list_api_result_jsonl_load = json.load(json_file)

df = pd.DataFrame(list_api_result_jsonl_load)
df

,prompt,completion,tokens
0,불고기용 고기 한우에요?,"'저는 인공지능 챗봇이며, 직접적으로 식품에 관한 정보를 가지고 있지 않습니다. 하...",193
1,쓰던 앱이 유료로 전환됐어,"'어떤 앱인지 모르기 때문에 정확한 답변을 드리기 어렵습니다. 하지만, 일반적으로 ...",288
2,여친이랑 다툼,'저는 인공지능이기 때문에 여친과의 다툼에 대해서는 조언을 드릴 수는 없습니다. 하...,153
3,술 먹고 싶어,"'술은 알코올이 함유된 음료수이며, 건강에 나쁜 영향을 미칠 수 있습니다. 따라서 ...",189
4,잊고싶다.,'저도 인공지능 엔진으로써 사용자의 개인정보나 감정을 침해할 수 없습니다. 그렇기 ...,147
...,...,...,...
11995,내일 개강총회인데 짝녀한테 어떻게 잘 보이지.,'제가 AI 모델이기 때문에 남성인점 참고 부탁드립니다. 다음과 같이 제안드립니다....,411
11996,역세권이에요?,"""저는 인공지능 언어모델이어서 지리적인 개념을 이해하는 데 한계가 있습니다. 반면,...",232
11997,암호화폐를 분산 저장하여 운영 관리 될 수 있도록 하는 인터넷상의 방식은?,'블록체인 기술을 사용하여 암호화폐를 분산 저장하여 운영 및 관리하는 것을 의미합니...,214
11998,가출해도 갈 데가 없어,"'그렇다면, 우선적으로 생활에 필요한 것들을 생각해보시는 것이 좋습니다. 가장 먼저...",473


In [11]:
## 전체 질문 답 쌍중 N개 질문만 추출하여 jsonl 형태로 저장
df_sample = df.sample(n=N_sample, random_state=230317)

# 500개의 질문만 선택
list_prompt = df_sample['prompt'].tolist()

list_ppo_prompt = []
for prompt in list_prompt:
    data={}
    data['prompt'] = prompt
    list_ppo_prompt.append(data)

print(list_ppo_prompt[:3])

with open(save_PPO_jsonl_path, "w", encoding='utf-8') as json_file:
    json.dump(list_ppo_prompt, json_file, indent=4, ensure_ascii=False)

[{'prompt': '번디는 자신이 탐정잡지, 범죄소설 그리고 성범죄 관련 실제 범죄 다큐멘터리들을 탐독했다고 누구에게 말했나?'}, {'prompt': '개포주공아파트는 몇 단지로 이루어져 있나?'}, {'prompt': '김영삼의 후보 시절 지역표심을 겨냥한 발언을 문제삼은 후보는?'}]


## END!!!